# UBIN parser

### Layer 1

In [ ]:
tables = camelot.read_pdf('/content/drive/MyDrive/SampleBankPdfs/AC/Union-AC-Statement-p7.pdf', pages="all", process_background=True)

# extract table header and convert it to df
table_header = pd.DataFrame([tables[0].df.iloc[0].tolist()], columns=tables[0].df.columns)

df_tables = []

for table in tables:
  df_tables.append(table.df.iloc[1:])

df_tables = pd.concat(df_tables, ignore_index=True)

df = pd.concat([table_header, df_tables], ignore_index=True)


# Set the first row as the new column names
df.columns = df.iloc[0]

# Drop the first row
df = df[1:].reset_index(drop=True)

# 4. Rename the columns
df.rename(columns={'Tran Id': 'id', 'Tran Date':'date', 'Remarks': 'description', 'Amount (Rs.)':'amount', 'Balance (Rs.)':'balance'}, inplace=True)

# Drop rows where specific columns have NaN values
df = df.dropna(subset=['description', 'amount', 'balance'])

# Dictionary of strings to replace
replace_dict = {
    '/': ' ',    # Replace forward slash with space
    '\n': ' ',   # Replace newline character with space
    '@': ' at '  # Replace '@' with ' at ' (optional example)
}

# Replace multiple strings using the dictionary
df['description'] = df['description'].replace(replace_dict, regex=True)

df['date'] = pd.to_datetime(df['date'], format='%d/%m/%Y').dt.strftime('%Y-%m-%d')

# Create the 'isDebit' column based on the presence of 'Dr' in the 'amount' column 
### (can add trxType as well)
df['isDebit'] = df['amount'].str.contains('Dr', regex=False)

# Remove any non-numeric characters (excluding periods and commas)
df['amount'] = df['amount'].str.replace(r'[^\d.]', '', regex=True).astype(float)

# Convert the cleaned amount column to numeric type
df['balance'] = pd.to_numeric(df['balance'])

display(df)

data_dict = df.to_dict(orient='records')
json_data = json.dumps(data_dict, ensure_ascii=False).replace('\\/', '/').replace('NaN', 'null')

print(json_data)

### Layer 2